## Creating and polling a website

This assignment trains you in 
1. installing and using a new Python framework
2. running Python code from the terminal
3. scrape web requests

The code in this assignment is based on the [`requests`](http://docs.python-requests.org/en/master/) module and the [flask](https://anaconda.org/anaconda/flask) framework (see links). Don't forget to use Python 3 for the assignment. Scary things might happen if you don't

## Part 1: Writing a server

The first part of the assignment is to write a *very* simple server. 
It needs to do three things:

1. Expose an HTTP POST method at `/`, that accepts and decodes a JSON data body. That is the only method you have to expose.
2. Get the request JSON data (using `request.get_json()`) and assume it's a dictionary. Now extract the value inside the dictionary under the key `password`. We will call this password for `client_password`.
3. Choose a two-letter password (in ASCII) that is fixed for the server. Just choose two random letters/numbers. We will call this password `SERVER_PASSWORD`. If the decoded value for the `password` field is the *same* as your chosen password (`client_password == SERVER_PASSWORD`), return an HTTP code `200 OK`. Otherwise return the HTTP code `403 Forbidden` status code. You don't have to return a body.

Hand in:
* The code for your server *copied into* a Jupyter Notebook
  * This should not be much more than 20 lines of code

In [5]:
#SERVER!!

# Installer flask: 
# $ conda install -c anaconda flask
# For at få programmet til at køre:
# $ export FLASK_APP=server.py
# $ export FLASK_ENV=development
# $ flask run
# * Running on http://127.0.0.1:5000/
# (eller localhost:5000/) -> starter serveren på localhost
# Brug eventuelt Postman til at køre requestene på
# POST - http://127.0.0.1:5000/ - body - raw - json - {"password": "acaca"} (kig på response kode)
# prøv med {"password": "ac"} og kig igen på response kode


#server.py preview for lazy people
from flask import Flask, request
import json

app = Flask(__name__)

SERVER_PASSWORD = 'ac'

@app.route('/', methods=['POST'])
def funct():
    data = request.get_data()
    my_dict = json.loads(data)
    CLIENT_PASSWORD = my_dict['password']
    if CLIENT_PASSWORD == SERVER_PASSWORD:
        response = app.response_class(status=200)
    else:
        response = app.response_class(status=403)
    return response

## Part 2: Hack the server!

Now we want to hack the server. But not just hack it once. We want to *automate* hacking the server!
Before you start on this part, make sure your server from part 1 is running.

1. Find a list of all the possible ASCII characters in Python (hint: look in the `string` module)
2. Generate *all possible combinations* of two ASCII characters and save them into the variable `
  * This will be 52 * 52 = 2704 elements. If you're smart you can create this as a generator
3. Write a method that takes a string `password` as its input, calls your server from part 1 (running on localhost), and *return* the status code of the request. Make sure that you:
  * Call the website at the correct port and URL with the correct HTTP verb
  * Inject data correctly. Remember that the body should be a `str` containing a JSON dictionary
4. Use your list of combinations from step 2 to call the sever 2704 times and find out when the status code it 200. When that happened you cracked the code!

Hand in:
  * The code for your hack in your Jupyter Notebook
  * The password you found

In [5]:
%%timeit
import multiprocessing
import string
import requests
import json


letters_and_numbers = list(string.ascii_letters)+list(string.digits)
# uden digits
# letters_and_numbers = list(string.ascii_letters)


string_list = []

for ln1 in letters_and_numbers:
    for ln2 in letters_and_numbers:
        string_list.append(str(ln1+ln2))

url = "http://localhost:5000/"

def call_server(password):
    jsonstr = json.dumps({ 'password' : password })
    r = requests.post(url, data=jsonstr)
    if r.status_code == 200:
        print('success')
        return password
    else:
        return ''    

right_pw = ''
for password in string_list[:]:
    p = multiprocessing.Process(target=call_server, args=(password,))
    p.start
    right_pw = call_server(password)
    print(password)
    if right_pw != '':
        break
        
print(right_pw)

aa
ab
ac
ad
ae
af
ag
ah
ai
aj
ak


KeyboardInterrupt: 

passwords and timeit: 
ac (3.05 s ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each))
0Z ()

## Part 3 (optional): Make it go fast!

This was probably pretty slow, because we made ~2704 sequential requests. Now implement this asynchronously by using
a pool like this:
```python
from multiprocessing import Pool
pool = Pool(processes=12)
```

Hand in:
* The code for parallelising your server hacking
* The runtimes of the sequential and parallel version using `%%timeit`